# Data Preprocessing

## Data Cleaning


* Drop columns that are not relevant to the problem. Example: URL, host picture etc.
* Find missing values for each column.
* Convert columns to their correct data type.
* One-hot-encode the categorical variables.

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import data
df = pd.read_csv('data/listings.csv') 
amenities_columns = pd.read_csv('data/amenities_columns.csv')

## Compute a new feature with distance from acropolis

In [3]:
from geopy.distance import geodesic
acropolis = (37.97171, 23.72603)
distances = []
df.longitude.dropna
df.latitude.dropna
for index,row in df.iterrows():
    listing = (row.latitude , row.longitude)
    distance = geodesic(acropolis, listing).kilometers
    distances.append(distance)

distances = [element * 1000 for element in distances]
df['distance'] = distances

## Keep features we want

In [4]:
df = df[['host_response_time','distance','host_response_rate','host_acceptance_rate', 'host_is_superhost', 'host_identity_verified' ,'neighbourhood_cleansed','room_type', 'accommodates', 'bathrooms_text', 'bedrooms','beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'has_availability', 'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'reviews_per_month' ]]

Categorical variables:
['host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_identity_verified', 'neighbourhood_cleansed', 'room_type', 'bathrooms_text', 'amenities', 'price', 'has_availability', 'instant_bookable']
ipykernel_launcher:32: FutureWarning: The default value of regex will change from True to False in a future version.


## Transform price, host response and acceptance rate

In [ ]:
df.host_response_rate = df.host_response_rate.replace({'\%':''}, regex = True).astype(float)
df.host_acceptance_rate = df.host_acceptance_rate.replace({'\%':''}, regex = True).astype(float)

df.price = df.price.replace({'\$':''}, regex = True)
df.price = df.price.replace({'\,':''}, regex = True).astype(float)

## Handle bathrooms feature and create shared feature

In [ ]:
df['number_of_baths'] = df.bathrooms_text.str.replace(r"[a-zA-Z]",'')
df['number_of_baths'] = df.number_of_baths.replace({'\-':np.nan}, regex = True)
df['number_of_baths'] = df['number_of_baths'].astype(float)
 
c = df.bathrooms_text.str.split(' ', expand = True)
df['shared_bath'] = c[1]

df.shared_bath = df.shared_bath == 'shared'
df = df.drop(columns = ['bathrooms_text', 'amenities'], axis = 1)

## Drop Unnamed column and add amenities columns

In [7]:
df.drop( ['Unnamed: 0'],axis=1,inplace=True)
df = pd.concat([df, amenities_columns], axis=1)

## FIX MISSING VALUES AND FILL THE T/F COLS
LIST 
* instant_bookable
* has_availability
* host_identity_verified
* host_is_superhost

In [11]:
df=df.dropna(subset=['instant_bookable','has_availability','host_identity_verified','host_is_superhost','beds'],how='any')

In [12]:
ColList=['instant_bookable','has_availability','host_identity_verified','host_is_superhost']
df['instant_bookable']=df['instant_bookable'].replace({'t':1,'f':0})
df['has_availability']=df['has_availability'].replace({'t':1,'f':0})
df['host_identity_verified']=df['host_identity_verified'].replace({'t':1,'f':0})
df['host_is_superhost']=df['host_is_superhost'].replace({'t':1,'f':0})
df['shared_bath']=df['shared_bath'].replace({True:1,False:0})       

## FIX THE HOST COLS
### Fill the NaN with strategy  


In [13]:
fill_strategy={
    'host_response_time': df['host_response_time'].mode()[0],
    'host_response_rate': df['host_response_rate'].mode()[0],
    'host_acceptance_rate': df['host_acceptance_rate'].mode()[0]
}
df=df.fillna(fill_strategy)

## FIX THE BEDROOMS NANS equals zero

In [14]:
df['bedrooms']=df['bedrooms'].fillna(0)

## FIX REVIEWS WITH MEAN 

In [15]:
df['review_scores_rating']=df['review_scores_rating'].fillna(df['review_scores_rating'].median())
df['review_scores_accuracy']=df['review_scores_accuracy'].fillna(df['review_scores_accuracy'].median())
df['review_scores_cleanliness']=df['review_scores_cleanliness'].fillna(df['review_scores_cleanliness'].median())
df['review_scores_checkin']=df['review_scores_checkin'].fillna(df['review_scores_checkin'].median())
df['review_scores_communication']=df['review_scores_communication'].fillna(df['review_scores_communication'].median())
df['review_scores_location']=df['review_scores_location'].fillna(df['review_scores_location'].median())
df['review_scores_value']=df['review_scores_value'].fillna(df['review_scores_value'].median())
df['reviews_per_month']=df['reviews_per_month'].fillna(df['reviews_per_month'].median())
df['number_of_baths']=df['number_of_baths'].fillna(df['number_of_baths'].median())

# ENCODING COLS

In [17]:
map_strategy={
    'within an hour': 0,
    'within a few hours': 1,
    'within a day': 2,
    'a few days or more': 3
}
df['host_response_time']=df['host_response_time'].map(map_strategy)

In [18]:
df_hot=pd.get_dummies(df['room_type'])
df = pd.concat([df, df_hot], axis=1)

In [19]:
df['Private room'].astype(float)
df['Entire home/apt'].astype(float)
df['Hotel room'].astype(float)
df['Shared room'].astype(float)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
9462    0.0
9463    0.0
9464    0.0
9465    0.0
9466    0.0
Name: Shared room, Length: 9409, dtype: float64

In [20]:
df.drop( ['room_type'],axis=1,inplace=True)

In [21]:
df_hot_two=pd.get_dummies(df['neighbourhood_cleansed'])
df_hot_two.astype(float)
df = pd.concat([df, df_hot_two], axis=1)

In [22]:
df.drop( ['neighbourhood_cleansed'],axis=1,inplace=True)

## drop some cols with high correlation 

In [26]:
df.drop(['bedrooms','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','reviews_per_month'], axis = 1, inplace=True)

## Filtering some outliers

In [ ]:
df = df.drop(df[df.price > 150].index)
df.drop(df[df.beds > 8].index, inplace=True)
df.drop(df[df.minimum_nights > 200].index, inplace=True)
df.drop(df[df.number_of_reviews > 600].index, inplace=True)
df.drop(df[df.maximum_nights > 1200].index, inplace=True)
df.drop(df[df.number_of_baths > 5].index, inplace=True)
index_names = df[ ((df['beds'] <= 1) & (df['price'] > 90))].index
df.drop(index_names, inplace = True)

## Moving column price, pos last 

In [27]:
col_target = ['price']
df = df[[c for c in df if c not in col_target] 
        + [c for c in col_target if c in df]]

## Export to csv

In [28]:
df.to_csv('ListingsFinal.csv', index=False)